# Importing Required Libraries

In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

# Loaing the File